In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings

from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import boxcox
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
                       
education = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [8]:
fill_list = ['ENROLL',
             'TOTAL_REVENUE',
             'FEDERAL_REVENUE',
             'STATE_REVENUE',
             'LOCAL_REVENUE',
             'TOTAL_EXPENDITURE',
             'INSTRUCTION_EXPENDITURE',
             'SUPPORT_SERVICES_EXPENDITURE',
             'OTHER_EXPENDITURE',
             'CAPITAL_OUTLAY_EXPENDITURE',
             'GRADES_PK_G',
             'GRADES_KG_G',
             'GRADES_4_G',
             'GRADES_8_G',
             'GRADES_12_G',
             'GRADES_1_8_G',
             'GRADES_9_12_G',
             'GRADES_ALL_G',
             'AVG_MATH_4_SCORE',
             'AVG_MATH_8_SCORE',
             'AVG_READING_4_SCORE',
             'AVG_READING_8_SCORE']

education2 = education.copy()

states = education2["STATE"].unique()

for state in states:
    education2.loc[education2["STATE"] == state, fill_list] = education2.loc[education2["STATE"] == state, fill_list].interpolate()
    
education3 = education2.copy()

for col in fill_list:
    education3.loc[:, col] = education2.loc[:, col].interpolate()

education3.dropna(inplace=True)

In [9]:
education3.isnull().sum()

PRIMARY_KEY                     0
STATE                           0
YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_PK_G                     0
GRADES_KG_G                     0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_1_8_G                    0
GRADES_9_12_G                   0
GRADES_ALL_G                    0
AVG_MATH_4_SCORE                0
AVG_MATH_8_SCORE                0
AVG_READING_4_SCORE             0
AVG_READING_8_SCORE             0
dtype: int64

# Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [10]:
education3['WEIGHTED_SCORES'] = (education3['GRADES_4_G'] * ((education3['AVG_MATH_4_SCORE'] + education3['AVG_READING_4_SCORE']) * 0.5)) + (
    education3['GRADES_8_G'] * ((education3['AVG_MATH_8_SCORE'] + education3['AVG_READING_8_SCORE']) * 0.5)) / (education3['GRADES_4_G'] + education3['GRADES_8_G'])

# What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [11]:
education3[['WEIGHTED_SCORES',
            'TOTAL_EXPENDITURE',
            'INSTRUCTION_EXPENDITURE',
            'SUPPORT_SERVICES_EXPENDITURE',
            'OTHER_EXPENDITURE',
            'CAPITAL_OUTLAY_EXPENDITURE']].corr()


,WEIGHTED_SCORES,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
WEIGHTED_SCORES,1.000000,0.746952,0.708935,0.746554,0.809500,0.846466
TOTAL_EXPENDITURE,0.746952,1.000000,0.992257,0.993688,0.945683,0.911341
INSTRUCTION_EXPENDITURE,0.708935,0.992257,1.000000,0.978819,0.911527,0.875006
SUPPORT_SERVICES_EXPENDITURE,0.746554,0.993688,0.978819,1.000000,0.952445,0.897020
OTHER_EXPENDITURE,0.809500,0.945683,0.911527,0.952445,1.000000,0.909893
CAPITAL_OUTLAY_EXPENDITURE,0.846466,0.911341,0.875006,0.897020,0.909893,1.000000


# Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [12]:
X = education3[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)


sklearn_pca = PCA(n_components=1)
education3["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)
#I'm not understanding what I need to do here

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94077648]


# If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

Because the first principal component explains around 94% of the variance, I would not use it instead of the expenditure variables.